In [1]:
import qualified DataFrame as D

train <- D.readCsv "./data/titanic_train.csv"
test <- D.readCsv "./data/titanic_test.csv"

In [2]:
import qualified Data.Text as T
import qualified DataFrame.Functions as F

import Data.Char
import DataFrame.Functions ((.=))
import DataFrame ((|>))

F.declareColumns train

combined = (train <> test)
            |> D.deriveMany
                [ -- Get only the alphanumeric characters of a ticket. 
                  F.name ticket .= F.whenPresent (T.filter isAlpha) (F.match "^([A-Za-z][A-Za-z0-9./]*)" ticket)
                  -- Match the title using a regex
                , F.name name   .= F.match "\\s*([A-Za-z]+)\\." name
                  -- take the first letter of the cabin
                , F.name cabin  .= F.whenPresent (T.take 1) cabin
                ]

In [3]:
import DataFrame.DecisionTree
import DataFrame.Functions ((.&&), (.==))
import System.Random

(train', validation) = combined
                |> D.take (D.nRows train)
                |> D.shuffle (mkStdGen 1894)
                |> D.randomSplit (mkStdGen 4232) 0.8

test' = D.drop (D.nRows train) combined

model = fitDecisionTree
                ( defaultTreeConfig
                    { maxTreeDepth = 5
                    , minSamplesSplit = 20
                    , minLeafSize = 5
                    }
                )
                survived
                ( train'
                    -- Unwrap maybes from passengers IDs
                    |> D.filterJust (F.name survived)
                    |> D.exclude [F.name passengerid]
                )

In [4]:
-- Let's inspect the learned model
model

(ifThenElse (eq (col @Maybe Text "Name") (lit (Just "Mr."))) (lit (0)) (ifThenElse (eq (col @Maybe Text "Cabin") (lit (Nothing))) (ifThenElse (eq (col @Text "Sex") (lit ("male"))) (ifThenElse (gt (toDouble (col @Int "SibSp")) (lit (1.0))) (lit (0)) (lit (1))) (ifThenElse (lt (toDouble (col @Int "Pclass")) (lit (3.0))) (lit (1)) (ifThenElse (eq (col @Maybe Text "Embarked") (lit (Just "S"))) (lit (0)) (lit (1))))) (lit (1))))

In [5]:
-- Column reference for predictions
prediction :: D.Expr Int
prediction = F.col @Int "prediction"

computeAccuracy :: D.DataFrame -> Double
computeAccuracy df =
    let
        cond c =  fromIntegral $ D.nRows (D.filterWhere (survived .== 1 .&& prediction .== 1) df)
        tp = fromIntegral $ D.nRows (D.filterWhere (survived .== 1 .&& prediction .== 1) df)
        tn = fromIntegral $ D.nRows (D.filterWhere (survived .== 0 .&& prediction .== 0) df)
        fp = fromIntegral $ D.nRows (D.filterWhere (survived .== 0 .&& prediction .== 1) df)
        fn = fromIntegral $ D.nRows (D.filterWhere (survived .== 1 .&& prediction .== 0) df)
     in
        (tp + tn) / (tp + tn + fp + fn)

In [6]:
putStrLn "Training accuracy: "
print $
    computeAccuracy
        (train' |> D.filterJust (F.name survived) |> D.derive (F.name prediction) model)

putStrLn "Validation accuracy: "
print $
    computeAccuracy
        ( validation
            |> D.filterJust (F.name survived)
            |> D.derive (F.name prediction) model
        )

test' |> D.derive (F.name survived) model
      |> D.take 10

Training accuracy:

0.8326359832635983

Validation accuracy:

0.7931034482758621

  ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  
| PassengerId<br>Int | Survived<br>Int | Pclass<br>Int | Name<br>Maybe Text | Sex<br>Text | Age<br>Maybe Double | SibSp<br>Int | Parch<br>Int | Ticket<br>Maybe Text | Fare<br>Maybe Double | Cabin<br>Maybe Text | Embarked<br>Maybe Text |
| -------------------|-----------------|---------------|--------------------|-------------|---------------------|--------------|--------------|----------------------|----------------------|---------------------|----------------------- |
| 892                | 0               | 3             | Just "Mr."         | male        | Just 34.5           | 0            | 0            | Nothing              | Just 7.8292          | Nothing             | Just "Q"               |
| 893                | 0               | 3             | Just "Mrs."        | female      | Just 47.0           | 1            | 0            | Nothing              | Just 7.0             | Nothing             | Just "S"               |
| 894                | 0               | 2             | Just "Mr."         | male        | Just 62.0           | 0            | 0            | Nothing              | Just 9.6875          | Nothing             | Just "Q"               |
| 895                | 0               | 3             | Just "Mr."         | male        | Just 27.0           | 0            | 0            | Nothing              | Just 8.6625          | Nothing             | Just "S"               |
| 896                | 0               | 3             | Just "Mrs."        | female      | Just 22.0           | 1            | 1            | Nothing              | Just 12.2875         | Nothing             | Just "S"               |
| 897                | 0               | 3             | Just "Mr."         | male        | Just 14.0           | 0            | 0            | Nothing              | Just 9.225           | Nothing             | Just "S"               |
| 898                | 1               | 3             | Just "Miss."       | female      | Just 30.0           | 0            | 0            | Nothing              | Just 7.6292          | Nothing             | Just "Q"               |
| 899                | 0               | 2             | Just "Mr."         | male        | Just 26.0           | 1            | 1            | Nothing              | Just 29.0            | Nothing             | Just "S"               |
| 900                | 1               | 3             | Just "Mrs."        | female      | Just 18.0           | 0            | 0            | Nothing              | Just 7.2292          | Nothing             | Just "C"               |
| 901                | 0               | 3             | Just "Mr."         | male        | Just 21.0           | 2            | 0            | Just "A"             | Just 24.15           | Nothing             | Just "S"               |
